# Processing Data

Dùng để chính thức đổi data thô thành data set, chia thành các mini batch đưa vào tập train

In [1]:
from lib_all import *

## I. Xây dựng dataset

Xây dựng lớp đại diện cho tập train/test

In [2]:
class MyDataset(data.Dataset):
    def __init__(self, list_img_path , list_anno_path, phase, DataTransform, InfoAnno ):
        """
        :param list_img_path:danh sách đường dẫn đến các bức ảnh
        :param list_anno_path:danh sách đường dẫn đến các file annotation
        :param phase:pha làm việc (train/val)
        :param DataTransform: transform anh
        :param InfoAnno: Thong tin tu annotation
        """
        self.list_img_path = list_img_path
        self.list_anno_path = list_anno_path
        self.phase = phase
        self.DataTransform = DataTransform
        self.InfoAnno = InfoAnno

    # Tim so luong buc anh ben trong dataset
    def __len__(self):
        return len(self.list_img_path)

    # lay tung phan tu ra
    def __getitem__(self, index):
        """
        :param index:
        :return: [1, num_obj]
        """
        img, ground_truth,height, width = self.pull_item(index)
        return img, ground_truth

    def pull_item(self, index):
        """
        :param index: vị trí của ảnh trong path
        :return: [1, num_obj, height, width]
        """
        # lay thong tin ve anh
        img_path = self.list_img_path[index]
        img = cv2.imread(img_path) #BGR
        height, width, channels = img.shape

        #lay thong tin ve annotation
        anno_path = self.list_anno_path[index]
        anno_info = self.InfoAnno(anno_path, width,height)

        #tien xu ly
        img, boxes, labels = self.DataTransform(img, self.phase, anno_info[:, :4], anno_info[:, 4])

        #doi thu tu cac kieu (BGR -> RGB) (height, width, channels ->channels, height, width) de phu hop voi cac ham cua pytorch
        img = torch.from_numpy(img[:,:,(2,1,0)]).permute(2,0,1)

        # gan gia tri cua bounding box, labels vao array
        ground_truth = np.hstack((boxes, np.expand_dims(labels, axis=1)))  #stack theo chieu ngang
        return img, ground_truth, height, width


## II. Dataloader
Dùng để load một lượng batch size từ MyDataset
Sử dụng lại hàm DataLoader trong thư viện data
Khác với bài toán classsification thì bài toán này không thể dùng collec_fc có sẵn được mà phải customize lại do 1 img có thể có nhiều object


In [3]:
def collate_fn(batch_img):
    """
    :param batch_img:gồm batch_size ảnh và annotation
    :return: batch_size, batch_size
    """
    annotations = []
    imgs = []

    for sample in batch_img:
        imgs.append(sample[0]) #sample[0] la thong so ve img
        annotations.append(torch.FloatTensor(sample[1]))

    # chuyen hinh dang imgs tu list[3, 300, 300]->tensor[batch_size, 3, 300, 300]
    imgs = torch.stack(imgs, dim=0)
    return imgs, annotations #Toàn bộ phải ở dạng tensor thì mới đưa được vào mạng ssd

 # Test

In [4]:
import nbimporter
from data.path import get_path
from data.processing_data import  DataTransform
from data.info_annotation import  InfoAnno

In [5]:
#annotation infomation
classes = ['aeroplane','bicycle','bird','boat','bottle','bus','car','cat','chair','cow','diningtable','dog','horse','motorbike','person','pottedplant','sheep','sofa','train','tvmonitor']

#get path
root_path = './data_set/VOCdevkit/VOC2012/'
train_images_path, train_annotations_path, val_images_path, val_annotations_path = get_path(root_path)

#prepare data transform
color_mean = (104, 117, 123)
input_size = 300

transform = DataTransform(input_size, color_mean)
InfoAnno = InfoAnno(classes)
train_dataset = MyDataset(train_images_path, train_annotations_path,phase="train", DataTransform =transform,InfoAnno= InfoAnno)
val_dataset = MyDataset(val_images_path, val_annotations_path,phase="train", DataTransform =transform,InfoAnno= InfoAnno)

Chiều dài phần tử

In [6]:
print(len(train_dataset))

5717


Phần tử đầu tiên


In [7]:
print(train_dataset.__getitem__(1))

(tensor([[[ -18.1495,  -18.8845,  -18.0329,  ...,   44.7338,   46.1155,
            46.8505],
         [ -17.2395,  -17.9745,  -17.6310,  ...,   47.9719,   48.8455,
            49.5805],
         [ -16.2995,  -15.7850,  -14.5666,  ...,   46.2167,   47.2412,
            49.8505],
         ...,
         [ -64.0495,  -80.7708,  -91.2941,  ...,  -55.4179,  -53.8035,
           -56.4495],
         [ -87.5895,  -93.3519,  -93.8550,  ...,  -38.6349,  -36.1791,
           -41.0595],
         [-102.1495,  -91.8595,  -87.5912,  ...,  -38.8245,  -37.9445,
           -40.1495]],

        [[ -39.1495,  -39.8845,  -38.4745,  ...,   41.7338,   43.1155,
            43.8505],
         [ -38.2395,  -38.9745,  -38.0726,  ...,   44.9719,   45.8455,
            46.5805],
         [ -37.2995,  -36.7850,  -35.0083,  ...,   43.2167,   44.2412,
            46.8505],
         ...,
         [ -58.7995,  -78.1300,  -90.2362,  ...,  -35.7933,  -34.6535,
           -37.2995],
         [ -86.9495,  -93.6454,  -95.45

C:\Users\DINH CHI CONG\OneDrive - Hanoi University of Science and Technology\Study\University\Machine Learning & Data Mining\Object_detection\sourc_code\utils\augmentations.py:241: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)


batch_size thường để là số chẵn vì khi đó nó sẽ đưa vào sử lý xong xong trong thread được

In [8]:
train_dataloader = data.DataLoader(train_dataset, batch_size = 4,shuffle=True,collate_fn=collate_fn)

# chuyển  thành iterator
batch_iter = iter(train_dataloader) # luot qua tung phan tu
images, annotations = next(batch_iter)

Chiều dài bộ dữ liệu giảm xuống 4 lần

In [9]:
print(len(train_dataloader))

1430


Mỗi iter bao gồm 4 ảnh


In [10]:
print(images.size())

torch.Size([4, 3, 300, 300])


Mỗi iter bao gồm 4 annotation

In [11]:
print("Chieu dai annotations {}".format(len(annotations)))

Chieu dai annotations 4


In [12]:
print(annotations[0].size())

torch.Size([2, 5])
